#  **Identificando discurso de ódio pelo LLaMa 3.1**

**Autores:** Luiz Fernando de Aguiar Lima, Hérique Costa Ribeiro de Lima, Adonias Caetano de Oliveira

## **Installation**

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## **Loading data**

In [2]:
import pandas as pd
import re
import random
from string import punctuation
import numpy as np

In [3]:
url = 'https://docs.google.com/spreadsheets/d/1qIKZNwZ99sxG8caWZBHCEHftp01EwqCs/edit?usp=sharing&ouid=114011919931967309237&rtpof=true&sd=true'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
dataset = pd.read_excel(read_url, index_col=0)

dataset['class'] = dataset['class'].astype(int)

# display the first 5 rows
dataset.head()

,tweet,class
0,"@jennybaquing so terms like honkey, cracker, p...",0
1,Wait did @kidfivevibes take my brownies??,2


## **Unsloth**

In [4]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.11 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 8192,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [6]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

## **Experimentos de identificação de ideação suicida**

In [7]:
def get_prompt(sentenca):
  return f"""
    Você é um modelo de IA treinado para classificar frases como:

    - "0" se a frase contém discurso de ódio (ataques direcionados a grupos com base em raça, religião, orientação sexual, etc.).
    - "1" se a frase contém linguagem ofensiva, mas não for discurso de ódio.
    - "2" se a frase não contém discurso de ódio nem linguagem ofensiva

    Regras:
    - Responda *apenas* com "0", "1" ou "2".
    - Não adicione explicações ou qualquer outro texto.

    Frase: "{sentenca}"
    """

In [ ]:

respostas_llm = {}

sentencas = dataset["tweet"].to_list()
targets = dataset["class"].to_list()

respostas_llm["text"] = sentencas
respostas_llm["target"] = targets
respostas_llm['predicted'] = []


for sentenca in sentencas:

  prompt = get_prompt(sentenca)

  prompt_tokenizado = tokenizer([prompt], return_tensors='pt').to('cuda')
  FastLanguageModel.for_inference(model)
  input_token_len = prompt_tokenizado['input_ids'].shape[1]
  output = model.generate(**prompt_tokenizado, max_new_tokens=512)
  # Seleciona apenas os tokens gerados (ignorando os tokens do prompt)
  resposta_ids = output[0, input_token_len:]
  # Decodifica apenas a nova resposta e remove espaços em branco extras
  resposta_texto = tokenizer.decode(resposta_ids, skip_special_tokens=True).strip()

  respostas_llm['predicted'].append(resposta_texto)

In [ ]:

from google.colab import files

df = pd.DataFrame(respostas_llm)
df.to_excel('respostas_llama3.xlsx', index=False)

files.download('respostas_llama3.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>